In [3]:
import init
import os, sys, logging
import cv2

sys.path.insert(0, 'lib')
from configs.faster.default_configs import config, update_config
import numpy as np

# get config
os.environ['PYTHONUNBUFFERED'] = '1'
os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = '0'
os.environ['MXNET_ENABLE_GPU_P2P'] = '0'
update_config('./configs/faster/res101_mx_3k.yml')

# for Superb AI Suite SDK
os.environ['LC_ALL'] = 'C.UTF-8'
os.environ['LANG'] = 'C.UTF-8'

import mxnet as mx
#from symbols import *

from demo.tictoc import tic, toc

/SNIPER/configs/faster/default_configs.py:169: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  exp_config = edict(yaml.load(f))


In [19]:
# get list of example images from local container

def get_image_list(image_root, class_names):
    files = []
    for idx, c in enumerate(class_names):
        files_c = [os.path.join(image_root, c, f) for f in os.listdir(os.path.join(image_root, c))
                   if os.path.isfile(os.path.join(image_root, c, f)) and f.split('.')[-1] in ['jpg', 'png']]
        files.append(files_c)
    return files

image_root = './demo/image'
class_names = ['bugatti', 'cleaner', 'gpu', 'sadcat', 'spiderman']

files = get_image_list(image_root, class_names)

print(', '.join(['{}: {} files'.format(name, len(files_c)) for name, files_c in zip(class_names, files)]))

bugatti: 115 files, cleaner: 111 files, gpu: 116 files, sadcat: 95 files, spiderman: 106 files


In [20]:
# randomly split example images into train set and val set

def random_split_into_train_val(files, num_val_per_class):
    train_files = []
    val_files = []

    for files_c in files:
        assert num_val_per_class < len(files_c)

        indices = list(range(len(files_c)))
        np.random.shuffle(indices)
        indices = indices[:num_val_per_class]
        indices.sort()

        for idx, f in enumerate(files_c):
            if idx in indices:
                val_files.append(f)
            else:
                train_files.append(f)

    return train_files, val_files

num_val_per_class = 50

train_files, val_files = random_split_into_train_val(files, num_val_per_class)

print('{} train files, {} val files'.format(len(train_files), len(val_files)))

293 train files, 250 val files


In [21]:
# authorize and initialize Superb AI Suite SDK

!printf "pingu\n69dba75b641ccde433e0512496f72b28\n" | spb configure

import spb
print(spb.__version__)

Suite Account Name: Access Key: Profile default is counfigured with account name 'pingu'.
0.0.34


In [25]:
# get project from Superb AI Suite

def get_spb_project(project_name):
    command = spb.Command(type='describe_project')

    projects, num_of_projects = spb.run(command=command, option={'name': project_name}, page=1, page_size=1)

    if num_of_projects == 0:
        raise Exception('Project {} not found'.format(project_name))
    assert num_of_projects == 1

    project = projects[0]
    print('Project ID: {}'.format(project.id))
    print('Project Name: {}'.format(project.name))
    return project

project = get_spb_project('SDK OD Test')

Project ID: 1ace2624-3395-433b-a67f-7a6a7b5de064
Project Name: SDK OD Test


In [24]:
# upload example images to Superb AI Suite

def upload_images_to_spb(project, dataset_name, files):
    command = spb.Command(type='create_data')

    for idx, f in enumerate(files):
        name = f.split('/')[-1]
        key = dataset_name + '_' + name
        option = {'file': f, 'file_name': name, 'dataset': dataset_name, 'data_key': key}

        try:
            spb.run(command=command, optional={'projectId': project.id}, option=option)
            print('Upload {}/{}: {} -> {}'.format(idx + 1, len(files), f, key))
        except Exception as e:
            print('[Exception {}] Upload {}/{}: {} -> {}'.format(str(e), idx + 1, len(files), f, key))

    print('Done')

spb_train_name = 'train_' + project.name.replace(' ', '_')
spb_val_name = 'val_' + project.name.replace(' ', '_')

upload_images_to_spb(project, spb_train_name, train_files)
upload_images_to_spb(project, spb_val_name, val_files)

Upload ./demo/image/bugatti/bugatti_78.jpg -> train_SDK_OD_Test_bugatti_78.jpg
Upload ./demo/image/bugatti/bugatti_3.jpg -> train_SDK_OD_Test_bugatti_3.jpg
Upload ./demo/image/bugatti/bugatti_63.jpg -> train_SDK_OD_Test_bugatti_63.jpg
Upload ./demo/image/bugatti/bugatti_1.jpg -> train_SDK_OD_Test_bugatti_1.jpg
Upload ./demo/image/bugatti/bugatti_84.jpg -> train_SDK_OD_Test_bugatti_84.jpg
Upload ./demo/image/bugatti/bugatti_18.jpg -> train_SDK_OD_Test_bugatti_18.jpg
Upload ./demo/image/bugatti/bugatti_29.jpg -> train_SDK_OD_Test_bugatti_29.jpg
Upload ./demo/image/bugatti/bugatti_64.jpg -> train_SDK_OD_Test_bugatti_64.jpg
Upload ./demo/image/bugatti/bugatti_98.jpg -> train_SDK_OD_Test_bugatti_98.jpg
Upload ./demo/image/bugatti/bugatti_111.jpg -> train_SDK_OD_Test_bugatti_111.jpg
Upload ./demo/image/bugatti/bugatti_83.jpg -> train_SDK_OD_Test_bugatti_83.jpg
Upload ./demo/image/bugatti/bugatti_32.jpg -> train_SDK_OD_Test_bugatti_32.jpg
Upload ./demo/image/bugatti/bugatti_46.jpg -> train_SD

Upload ./demo/image/cleaner/cleaner_44.jpg -> train_SDK_OD_Test_cleaner_44.jpg
Upload ./demo/image/cleaner/cleaner_83.jpg -> train_SDK_OD_Test_cleaner_83.jpg
Upload ./demo/image/cleaner/cleaner_63.jpg -> train_SDK_OD_Test_cleaner_63.jpg
Upload ./demo/image/cleaner/cleaner_93.jpg -> train_SDK_OD_Test_cleaner_93.jpg
Upload ./demo/image/cleaner/cleaner_106.jpg -> train_SDK_OD_Test_cleaner_106.jpg
Upload ./demo/image/cleaner/cleaner_28.jpg -> train_SDK_OD_Test_cleaner_28.jpg
Upload ./demo/image/cleaner/cleaner_29.jpg -> train_SDK_OD_Test_cleaner_29.jpg
Upload ./demo/image/cleaner/cleaner_15.jpg -> train_SDK_OD_Test_cleaner_15.jpg
Upload ./demo/image/cleaner/cleaner_69.jpg -> train_SDK_OD_Test_cleaner_69.jpg
Upload ./demo/image/cleaner/cleaner_52.jpg -> train_SDK_OD_Test_cleaner_52.jpg
Upload ./demo/image/cleaner/cleaner_31.jpg -> train_SDK_OD_Test_cleaner_31.jpg
Upload ./demo/image/cleaner/cleaner_115.jpg -> train_SDK_OD_Test_cleaner_115.jpg
Upload ./demo/image/cleaner/cleaner_30.jpg -> tr

Upload ./demo/image/sadcat/sadcat_49.jpg -> train_SDK_OD_Test_sadcat_49.jpg
Upload ./demo/image/sadcat/sadcat_77.jpg -> train_SDK_OD_Test_sadcat_77.jpg
Upload ./demo/image/sadcat/sadcat_47.jpg -> train_SDK_OD_Test_sadcat_47.jpg
Upload ./demo/image/sadcat/sadcat_85.jpg -> train_SDK_OD_Test_sadcat_85.jpg
Upload ./demo/image/sadcat/sadcat_24.jpg -> train_SDK_OD_Test_sadcat_24.jpg
Upload ./demo/image/sadcat/sadcat_23.jpg -> train_SDK_OD_Test_sadcat_23.jpg
Upload ./demo/image/sadcat/sadcat_29.jpg -> train_SDK_OD_Test_sadcat_29.jpg
Upload ./demo/image/sadcat/sadcat_19.jpg -> train_SDK_OD_Test_sadcat_19.jpg
Upload ./demo/image/sadcat/sadcat_75.jpg -> train_SDK_OD_Test_sadcat_75.jpg
Upload ./demo/image/sadcat/sadcat_101.jpg -> train_SDK_OD_Test_sadcat_101.jpg
Upload ./demo/image/sadcat/sadcat_57.jpg -> train_SDK_OD_Test_sadcat_57.jpg
Upload ./demo/image/sadcat/sadcat_67.jpg -> train_SDK_OD_Test_sadcat_67.jpg
Upload ./demo/image/sadcat/sadcat_79.jpg -> train_SDK_OD_Test_sadcat_79.jpg
Upload ./d

Upload ./demo/image/bugatti/bugatti_59.jpg -> val_SDK_OD_Test_bugatti_59.jpg
Upload ./demo/image/bugatti/bugatti_53.jpg -> val_SDK_OD_Test_bugatti_53.jpg
Upload ./demo/image/bugatti/bugatti_68.jpg -> val_SDK_OD_Test_bugatti_68.jpg
Upload ./demo/image/bugatti/bugatti_88.jpg -> val_SDK_OD_Test_bugatti_88.jpg
Upload ./demo/image/bugatti/bugatti_75.jpg -> val_SDK_OD_Test_bugatti_75.jpg
Upload ./demo/image/bugatti/bugatti_91.jpg -> val_SDK_OD_Test_bugatti_91.jpg
Upload ./demo/image/bugatti/bugatti_69.jpg -> val_SDK_OD_Test_bugatti_69.jpg
Upload ./demo/image/bugatti/bugatti_26.jpg -> val_SDK_OD_Test_bugatti_26.jpg
Upload ./demo/image/bugatti/bugatti_14.jpg -> val_SDK_OD_Test_bugatti_14.jpg
Upload ./demo/image/bugatti/bugatti_25.jpg -> val_SDK_OD_Test_bugatti_25.jpg
Upload ./demo/image/bugatti/bugatti_27.jpg -> val_SDK_OD_Test_bugatti_27.jpg
Upload ./demo/image/bugatti/bugatti_76.jpg -> val_SDK_OD_Test_bugatti_76.jpg
Upload ./demo/image/bugatti/bugatti_8.jpg -> val_SDK_OD_Test_bugatti_8.jpg
U

Upload ./demo/image/gpu/gpu_114.jpg -> val_SDK_OD_Test_gpu_114.jpg
Upload ./demo/image/gpu/gpu_7.jpg -> val_SDK_OD_Test_gpu_7.jpg
Upload ./demo/image/gpu/gpu_93.jpg -> val_SDK_OD_Test_gpu_93.jpg
Upload ./demo/image/gpu/gpu_60.jpg -> val_SDK_OD_Test_gpu_60.jpg
Upload ./demo/image/gpu/gpu_51.jpg -> val_SDK_OD_Test_gpu_51.jpg
Upload ./demo/image/gpu/gpu_50.jpg -> val_SDK_OD_Test_gpu_50.jpg
Upload ./demo/image/gpu/gpu_108.jpg -> val_SDK_OD_Test_gpu_108.jpg
Upload ./demo/image/gpu/gpu_41.jpg -> val_SDK_OD_Test_gpu_41.jpg
Upload ./demo/image/gpu/gpu_12.jpg -> val_SDK_OD_Test_gpu_12.jpg
Upload ./demo/image/gpu/gpu_94.jpg -> val_SDK_OD_Test_gpu_94.jpg
Upload ./demo/image/sadcat/sadcat_97.jpg -> val_SDK_OD_Test_sadcat_97.jpg
Upload ./demo/image/sadcat/sadcat_6.jpg -> val_SDK_OD_Test_sadcat_6.jpg
Upload ./demo/image/sadcat/sadcat_63.jpg -> val_SDK_OD_Test_sadcat_63.jpg
Upload ./demo/image/sadcat/sadcat_39.jpg -> val_SDK_OD_Test_sadcat_39.jpg
Upload ./demo/image/sadcat/sadcat_59.jpg -> val_SDK_OD

Upload ./demo/image/spiderman/spiderman_73.jpg -> val_SDK_OD_Test_spiderman_73.jpg
Upload ./demo/image/spiderman/spiderman_62.jpg -> val_SDK_OD_Test_spiderman_62.jpg
Upload ./demo/image/spiderman/spiderman_30.jpg -> val_SDK_OD_Test_spiderman_30.jpg
Done


In [64]:
# get uploaded data info from Superb AI Suite

def get_spb_data(project):
    command = spb.Command(type='describe_label')
    option = {'project_id': project.id}
    page_size = 10

    try:
        data, num_data = spb.run(command=command, option=option, page=1, page_size=page_size)
        num_pages = int(np.ceil(float(num_data) / page_size))
        print('Load page {}/{}'.format(1, num_pages))
    except Excetion as e:
        print('[Exception {}] Load page {}'.format(str(e), 1))
        return []

    for idx in range(1, num_pages):
        try:
            data_pg, _ = spb.run(command=command, option=option, page=idx+1, page_size=page_size)
            data = data + data_pg
            print('Load page {}/{}'.format(idx + 1, num_pages))
        except Excetion as e:
            print('[Exception {}] Load page {}/{}'.format(str(e), idx + 1, num_pages))

    print('Done')
    return data

spb_data = get_spb_data(project)

Load page 1/55
Load page 2/55
Load page 3/55
Load page 4/55
Load page 5/55
Load page 6/55
Load page 7/55
Load page 8/55
Load page 9/55
Load page 10/55
Load page 11/55
Load page 12/55
Load page 13/55
Load page 14/55
Load page 15/55
Load page 16/55
Load page 17/55
Load page 18/55
Load page 19/55
Load page 20/55
Load page 21/55
Load page 22/55
Load page 23/55
Load page 24/55
Load page 25/55
Load page 26/55
Load page 27/55
Load page 28/55
Load page 29/55
Load page 30/55
Load page 31/55
Load page 32/55
Load page 33/55
Load page 34/55
Load page 35/55
Load page 36/55
Load page 37/55
Load page 38/55
Load page 39/55
Load page 40/55
Load page 41/55
Load page 42/55
Load page 43/55
Load page 44/55
Load page 45/55
Load page 46/55
Load page 47/55
Load page 48/55
Load page 49/55
Load page 50/55
Load page 51/55
Load page 52/55
Load page 53/55
Load page 54/55
Load page 55/55
Done


In [84]:
# annotate image category to uploaded data in Superb AI Suite

def annotate_image_category(project, spb_data):
    command = spb.Command(type='update_label')
    class_name_to_spb_id = {c['name']: c['id']
                            for c in project.label_interface['categorization']['word_map'] if c['id'] != 'root'}

    for idx, d in enumerate(spb_data):
        class_name = d.data_key.split('_')[-2]
        d.result = {'objects': [], 'categorization': {'value': [class_name_to_spb_id[class_name]]}}

        try:
            d = spb.run(command=command, option=d)
            print('Annotate {}/{}: {} -> {}'.format(idx + 1, len(spb_data), class_name, d.data_key))
        except Exception as e:
            print('[Exception {}] Annotate {}/{}: {} -> {}'.format(str(e), idx + 1, len(spb_data), class_name, d.data_key))

    print('Done')

annotate_image_category(project, spb_data)

Annotate 1/542: bugatti -> train_SDK_OD_Test_bugatti_78.jpg
Annotate 2/542: bugatti -> train_SDK_OD_Test_bugatti_3.jpg
Annotate 3/542: bugatti -> train_SDK_OD_Test_bugatti_63.jpg
Annotate 4/542: bugatti -> train_SDK_OD_Test_bugatti_1.jpg
Annotate 5/542: bugatti -> train_SDK_OD_Test_bugatti_84.jpg
Annotate 6/542: bugatti -> train_SDK_OD_Test_bugatti_18.jpg
Annotate 7/542: bugatti -> train_SDK_OD_Test_bugatti_29.jpg
Annotate 8/542: bugatti -> train_SDK_OD_Test_bugatti_64.jpg
Annotate 9/542: bugatti -> train_SDK_OD_Test_bugatti_98.jpg
Annotate 10/542: bugatti -> train_SDK_OD_Test_bugatti_111.jpg
Annotate 11/542: bugatti -> train_SDK_OD_Test_bugatti_83.jpg
Annotate 12/542: bugatti -> train_SDK_OD_Test_bugatti_32.jpg
Annotate 13/542: bugatti -> train_SDK_OD_Test_bugatti_46.jpg
Annotate 14/542: bugatti -> train_SDK_OD_Test_bugatti_12.jpg
Annotate 15/542: bugatti -> train_SDK_OD_Test_bugatti_85.jpg
Annotate 16/542: bugatti -> train_SDK_OD_Test_bugatti_67.jpg
Annotate 17/542: bugatti -> train_

Annotate 136/542: gpu -> train_SDK_OD_Test_gpu_92.jpg
Annotate 137/542: gpu -> train_SDK_OD_Test_gpu_65.jpg
Annotate 138/542: gpu -> train_SDK_OD_Test_gpu_70.jpg
Annotate 139/542: gpu -> train_SDK_OD_Test_gpu_24.jpg
Annotate 140/542: gpu -> train_SDK_OD_Test_gpu_69.jpg
Annotate 141/542: gpu -> train_SDK_OD_Test_gpu_48.jpg
Annotate 142/542: gpu -> train_SDK_OD_Test_gpu_87.jpg
Annotate 143/542: gpu -> train_SDK_OD_Test_gpu_9.jpg
Annotate 144/542: gpu -> train_SDK_OD_Test_gpu_113.jpg
Annotate 145/542: gpu -> train_SDK_OD_Test_gpu_68.jpg
Annotate 146/542: gpu -> train_SDK_OD_Test_gpu_89.jpg
Annotate 147/542: gpu -> train_SDK_OD_Test_gpu_85.jpg
Annotate 148/542: gpu -> train_SDK_OD_Test_gpu_106.jpg
Annotate 149/542: gpu -> train_SDK_OD_Test_gpu_88.jpg
Annotate 150/542: gpu -> train_SDK_OD_Test_gpu_46.jpg
Annotate 151/542: gpu -> train_SDK_OD_Test_gpu_42.jpg
Annotate 152/542: gpu -> train_SDK_OD_Test_gpu_72.jpg
Annotate 153/542: gpu -> train_SDK_OD_Test_gpu_83.jpg
Annotate 154/542: gpu -> tr

Annotate 275/542: spiderman -> train_SDK_OD_Test_spiderman_5.jpg
Annotate 276/542: spiderman -> train_SDK_OD_Test_spiderman_37.jpg
Annotate 277/542: spiderman -> train_SDK_OD_Test_spiderman_79.jpg
Annotate 278/542: spiderman -> train_SDK_OD_Test_spiderman_74.jpg
Annotate 279/542: spiderman -> train_SDK_OD_Test_spiderman_1.jpg
Annotate 280/542: spiderman -> train_SDK_OD_Test_spiderman_11.jpg
Annotate 281/542: spiderman -> train_SDK_OD_Test_spiderman_4.jpg
Annotate 282/542: spiderman -> train_SDK_OD_Test_spiderman_90.jpg
Annotate 283/542: spiderman -> train_SDK_OD_Test_spiderman_35.jpg
Annotate 284/542: spiderman -> train_SDK_OD_Test_spiderman_59.jpg
Annotate 285/542: spiderman -> train_SDK_OD_Test_spiderman_16.jpg
Annotate 286/542: spiderman -> train_SDK_OD_Test_spiderman_12.jpg
Annotate 287/542: spiderman -> train_SDK_OD_Test_spiderman_21.jpg
Annotate 288/542: spiderman -> train_SDK_OD_Test_spiderman_102.jpg
Annotate 289/542: spiderman -> train_SDK_OD_Test_spiderman_18.jpg
Annotate 290

Annotate 413/542: gpu -> val_SDK_OD_Test_gpu_97.jpg
Annotate 414/542: gpu -> val_SDK_OD_Test_gpu_117.jpg
Annotate 415/542: gpu -> val_SDK_OD_Test_gpu_40.jpg
Annotate 416/542: gpu -> val_SDK_OD_Test_gpu_107.jpg
Annotate 417/542: gpu -> val_SDK_OD_Test_gpu_53.jpg
Annotate 418/542: gpu -> val_SDK_OD_Test_gpu_81.jpg
Annotate 419/542: gpu -> val_SDK_OD_Test_gpu_105.jpg
Annotate 420/542: gpu -> val_SDK_OD_Test_gpu_66.jpg
Annotate 421/542: gpu -> val_SDK_OD_Test_gpu_5.jpg
Annotate 422/542: gpu -> val_SDK_OD_Test_gpu_37.jpg
Annotate 423/542: gpu -> val_SDK_OD_Test_gpu_75.jpg
Annotate 424/542: gpu -> val_SDK_OD_Test_gpu_0.jpg
Annotate 425/542: gpu -> val_SDK_OD_Test_gpu_61.jpg
Annotate 426/542: gpu -> val_SDK_OD_Test_gpu_96.jpg
Annotate 427/542: gpu -> val_SDK_OD_Test_gpu_63.jpg
Annotate 428/542: gpu -> val_SDK_OD_Test_gpu_21.jpg
Annotate 429/542: gpu -> val_SDK_OD_Test_gpu_98.jpg
Annotate 430/542: gpu -> val_SDK_OD_Test_gpu_104.jpg
Annotate 431/542: gpu -> val_SDK_OD_Test_gpu_36.jpg
Annotate 4

In [66]:
# load and transform images from Superb AI Suite

import skimage.io
from demo.image import resize, transform

def load_and_transform_spb_images(project, spb_data, class_names):
    images = []
    im_infos = []
    mx_data = []
    labels = []
    ref_spb_data = []

    class_spb_id_to_name = {c['id']: c['name']
                            for c in project.label_interface['categorization']['word_map'] if c['id'] != 'root'}
    class_name_to_label = {name: idx for idx, name in enumerate(class_names)}

    print('Load and transform images...')
    tic()

    for idx, d in enumerate(spb_data):
        try:
            im = skimage.io.imread(d.data_url)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

            class_spb_id = d.result['categorization']['value'][0]
            class_name = class_spb_id_to_name[class_spb_id]
            label = class_name_to_label[class_name]

        except Exception as e:
            print('[Exception {}] Load and transform {}'.format(str(e), d.data_key))
            continue

        target_size = config.TEST.SCALES[0][0]
        max_size = config.TEST.SCALES[0][1]

        im, im_scale = resize(im, target_size, max_size, stride=config.network.RPN_FEAT_STRIDE)
        im_tensor = transform(im, config.network.PIXEL_MEANS)
        im_info = np.array([[im_tensor.shape[2], im_tensor.shape[3], im_scale]], dtype=np.float32)

        images.append(im)
        im_infos.append(im_info)
        mx_data.append([mx.nd.array(im_tensor), mx.nd.array(im_info)])
        labels.append(label)
        ref_spb_data.append(d)

        if (idx + 1) % 100 == 0:
            print('Load and transform {}/{}: {:.4f} seconds spent.'.format(idx + 1, len(spb_data), toc()))

    print('Done')
    return images, im_infos, mx_data, labels, ref_spb_data

images, im_infos, mx_data, labels, ref_spb_data = load_and_transform_spb_images(project, spb_data, class_names)

Load and transform images...
Load and transform 100/542: 17.1483 seconds spent.
Load and transform 200/542: 32.7651 seconds spent.
Load and transform 300/542: 48.6167 seconds spent.
Load and transform 400/542: 65.2874 seconds spent.
Load and transform 500/542: 80.0002 seconds spent.
Done


In [44]:
# build feature extractor

from demo.module import MutableModule
from demo.load_model import load_param
from symbols.faster.resnet_mx_101_e2e_3k_demo import resnet_mx_101_e2e_3k_demo

def build_feat_extractor(config, arch, ckpt, dummy_mx_data):
    # calculate maximum input size
    max_im_h = max([v[0] for v in config.TEST.SCALES])
    max_im_w = max([v[1] for v in config.TEST.SCALES])
    max_data_shape = [('data', (1, 3, max_im_h, max_im_w))]

    # build architecture
    feat_extractor = MutableModule(arch.get_symbol_rcnn(config, is_train=False),
                                   data_names=['data', 'im_info'], label_names=[],
                                   context=[mx.gpu(0)],
                                   max_data_shapes=[max_data_shape])

    # initialize parameters
    feat_extractor.bind([[('data', dummy_mx_data[0].shape), ('im_info', dummy_mx_data[1].shape)]],
                        [None], for_training=False)

    # load pre-trained weights
    arg_params, aux_params = load_param(ckpt, config.TEST.TEST_EPOCH, convert=True, process=True)
    feat_extractor.init_params(arg_params=arg_params, aux_params=aux_params)

    return feat_extractor

feat_extractor_arch = resnet_mx_101_e2e_3k_demo()
feat_extractor_ckpt = './output/chips_resnet101_3k/res101_mx_3k/fall11_whole/CRCNN'

feat_extractor = build_feat_extractor(config, feat_extractor_arch, feat_extractor_ckpt, mx_data[0])

In [67]:
# image-level feature extraction

def extract_im_feats(feat_extractor, mx_data):
    im_feats = []

    print("Extract image-level features...")
    tic()

    for idx, d in enumerate(mx_data):
        # batch of size 1
        x = mx.io.DataBatch(data=[d], label=[], pad=0, index=idx,
                            provide_data=[[('data', d[0].shape), ('im_info', d[1].shape)]],
                            provide_label=[None])

        # global average pooling of conv feature
        feat_extractor.forward(x)
        conv5_feat = feat_extractor.get_outputs()[4]
        pooled_feat = mx.ndarray.Pooling(data=conv5_feat, pool_type='avg', global_pool=True, kernel=(7, 7))
        im_feats.append(pooled_feat.reshape((1, -1)).asnumpy())

        if (idx + 1) % 100 == 0:
            print('Image-level feature extraction {}/{}: {:.4f} seconds spent.'.format(idx + 1, len(mx_data), toc()))

    print('Done')
    return im_feats

im_feats = extract_im_feats(feat_extractor, mx_data)

Extract image-level features...
Image-level feature extraction 100/542: 5.2283 seconds spent.
Image-level feature extraction 200/542: 10.9699 seconds spent.
Image-level feature extraction 300/542: 16.3819 seconds spent.
Image-level feature extraction 400/542: 22.5864 seconds spent.
Image-level feature extraction 500/542: 28.3669 seconds spent.
Done


In [49]:
# split train val sets

def split_train_val(elems, spb_data, spb_train_name, spb_val_name):
    train_elems = [elem for elem, d in zip(elems, spb_data) if d.dataset == spb_train_name]
    val_elems = [elem for elem, d in zip(elems, spb_data) if d.dataset == spb_val_name]
    return train_elems, val_elems

train_im_feats, val_im_feats = split_train_val(im_feats, spb_data, spb_train_name, spb_val_name)
train_labels, val_labels = split_train_val(labels, spb_data, spb_train_name, spb_val_name)

In [50]:
# build linear classifier that predicts class scores for images or RoIs

def build_linear_classifier(logger, num_of_classes):
    x = mx.sym.Variable('data')
    y = mx.sym.Variable("softmax_label")
    logit = mx.sym.FullyConnected(data=x, name='fc1', num_hidden=num_of_classes)
    softmax = mx.sym.SoftmaxOutput(data=logit, label=y, name='softmax')

    model = mx.mod.Module(symbol=softmax, data_names=['data'], label_names=['softmax_label'],
                          logger=logger, context=mx.gpu())
    return model

logger = logging.getLogger()
logger.setLevel(logging.INFO)

linear_classifier = build_linear_classifier(logger, len(class_names))

In [51]:
# train inear classifier

def train_linear_classifier(linear_classifier, train_im_feats, train_labels, val_im_feats, val_labels,
                           batch_size, learning_rate, momentum, num_epoch):

    train_iterator = mx.io.NDArrayIter(np.concatenate(train_im_feats),
                                       np.array(train_labels),
                                       batch_size, shuffle=True)

    val_iterator = mx.io.NDArrayIter(np.concatenate(val_im_feats),
                                       np.array(val_labels),
                                       batch_size, shuffle=False)

    linear_classifier.fit(train_iterator, val_iterator, optimizer='sgd',
                          optimizer_params={'learning_rate':learning_rate, 'momentum': momentum},
                          num_epoch=num_epoch, eval_metric='acc',
                          batch_end_callback = mx.callback.Speedometer(batch_size, 2))

batch_size = 30
learning_rate = 0.001
momentum = 0.9
num_epoch = 250

train_linear_classifier(linear_classifier, train_im_feats, train_labels, val_im_feats, val_labels,
                       batch_size, learning_rate, momentum, num_epoch)

INFO:root:Epoch[0] Batch [2]	Speed: 31007.67 samples/sec	accuracy=0.388889
INFO:root:Epoch[0] Batch [4]	Speed: 22760.08 samples/sec	accuracy=0.650000
INFO:root:Epoch[0] Batch [6]	Speed: 21048.70 samples/sec	accuracy=0.783333
INFO:root:Epoch[0] Batch [8]	Speed: 19316.72 samples/sec	accuracy=0.950000
INFO:root:Epoch[0] Train-accuracy=0.866667
INFO:root:Epoch[0] Time cost=0.020
INFO:root:Epoch[0] Validation-accuracy=0.959259
INFO:root:Epoch[1] Batch [2]	Speed: 28111.96 samples/sec	accuracy=0.988889
INFO:root:Epoch[1] Batch [4]	Speed: 20355.76 samples/sec	accuracy=0.933333
INFO:root:Epoch[1] Batch [6]	Speed: 18279.82 samples/sec	accuracy=0.933333
INFO:root:Epoch[1] Batch [8]	Speed: 16082.45 samples/sec	accuracy=0.966667
INFO:root:Epoch[1] Train-accuracy=0.966667
INFO:root:Epoch[1] Time cost=0.021
INFO:root:Epoch[1] Validation-accuracy=0.981481
INFO:root:Epoch[2] Batch [2]	Speed: 26814.94 samples/sec	accuracy=1.000000
INFO:root:Epoch[2] Batch [4]	Speed: 20229.76 samples/sec	accuracy=0.98333

INFO:root:Epoch[19] Batch [4]	Speed: 18341.10 samples/sec	accuracy=0.983333
INFO:root:Epoch[19] Batch [6]	Speed: 19380.69 samples/sec	accuracy=1.000000
INFO:root:Epoch[19] Batch [8]	Speed: 16105.10 samples/sec	accuracy=1.000000
INFO:root:Epoch[19] Train-accuracy=1.000000
INFO:root:Epoch[19] Time cost=0.021
INFO:root:Epoch[19] Validation-accuracy=0.985185
INFO:root:Epoch[20] Batch [2]	Speed: 26961.46 samples/sec	accuracy=1.000000
INFO:root:Epoch[20] Batch [4]	Speed: 19725.52 samples/sec	accuracy=0.983333
INFO:root:Epoch[20] Batch [6]	Speed: 20763.88 samples/sec	accuracy=1.000000
INFO:root:Epoch[20] Batch [8]	Speed: 15899.56 samples/sec	accuracy=1.000000
INFO:root:Epoch[20] Train-accuracy=1.000000
INFO:root:Epoch[20] Time cost=0.021
INFO:root:Epoch[20] Validation-accuracy=0.985185
INFO:root:Epoch[21] Batch [2]	Speed: 26780.70 samples/sec	accuracy=1.000000
INFO:root:Epoch[21] Batch [4]	Speed: 20488.34 samples/sec	accuracy=0.983333
INFO:root:Epoch[21] Batch [6]	Speed: 19971.29 samples/sec	

INFO:root:Epoch[38] Batch [4]	Speed: 23974.30 samples/sec	accuracy=0.983333
INFO:root:Epoch[38] Batch [6]	Speed: 22610.80 samples/sec	accuracy=1.000000
INFO:root:Epoch[38] Batch [8]	Speed: 19128.78 samples/sec	accuracy=1.000000
INFO:root:Epoch[38] Train-accuracy=1.000000
INFO:root:Epoch[38] Time cost=0.019
INFO:root:Epoch[38] Validation-accuracy=0.988889
INFO:root:Epoch[39] Batch [2]	Speed: 33572.34 samples/sec	accuracy=1.000000
INFO:root:Epoch[39] Batch [4]	Speed: 23235.00 samples/sec	accuracy=0.983333
INFO:root:Epoch[39] Batch [6]	Speed: 21199.41 samples/sec	accuracy=1.000000
INFO:root:Epoch[39] Batch [8]	Speed: 17419.41 samples/sec	accuracy=1.000000
INFO:root:Epoch[39] Train-accuracy=1.000000
INFO:root:Epoch[39] Time cost=0.019
INFO:root:Epoch[39] Validation-accuracy=0.988889
INFO:root:Epoch[40] Batch [2]	Speed: 30533.64 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Batch [4]	Speed: 21577.49 samples/sec	accuracy=0.983333
INFO:root:Epoch[40] Batch [6]	Speed: 22292.34 samples/sec	

INFO:root:Epoch[57] Batch [4]	Speed: 23862.91 samples/sec	accuracy=0.983333
INFO:root:Epoch[57] Batch [6]	Speed: 22702.59 samples/sec	accuracy=1.000000
INFO:root:Epoch[57] Batch [8]	Speed: 17173.35 samples/sec	accuracy=1.000000
INFO:root:Epoch[57] Train-accuracy=1.000000
INFO:root:Epoch[57] Time cost=0.021
INFO:root:Epoch[57] Validation-accuracy=0.985185
INFO:root:Epoch[58] Batch [2]	Speed: 30331.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[58] Batch [4]	Speed: 21704.03 samples/sec	accuracy=0.983333
INFO:root:Epoch[58] Batch [6]	Speed: 23353.59 samples/sec	accuracy=1.000000
INFO:root:Epoch[58] Batch [8]	Speed: 18023.22 samples/sec	accuracy=1.000000
INFO:root:Epoch[58] Train-accuracy=1.000000
INFO:root:Epoch[58] Time cost=0.021
INFO:root:Epoch[58] Validation-accuracy=0.985185
INFO:root:Epoch[59] Batch [2]	Speed: 30258.30 samples/sec	accuracy=1.000000
INFO:root:Epoch[59] Batch [4]	Speed: 19848.43 samples/sec	accuracy=0.983333
INFO:root:Epoch[59] Batch [6]	Speed: 22235.22 samples/sec	

INFO:root:Epoch[76] Batch [4]	Speed: 15696.27 samples/sec	accuracy=1.000000
INFO:root:Epoch[76] Batch [6]	Speed: 18413.57 samples/sec	accuracy=1.000000
INFO:root:Epoch[76] Batch [8]	Speed: 14666.25 samples/sec	accuracy=1.000000
INFO:root:Epoch[76] Train-accuracy=1.000000
INFO:root:Epoch[76] Time cost=0.024
INFO:root:Epoch[76] Validation-accuracy=0.985185
INFO:root:Epoch[77] Batch [2]	Speed: 27896.93 samples/sec	accuracy=1.000000
INFO:root:Epoch[77] Batch [4]	Speed: 21620.12 samples/sec	accuracy=1.000000
INFO:root:Epoch[77] Batch [6]	Speed: 21568.24 samples/sec	accuracy=1.000000
INFO:root:Epoch[77] Batch [8]	Speed: 17133.59 samples/sec	accuracy=1.000000
INFO:root:Epoch[77] Train-accuracy=1.000000
INFO:root:Epoch[77] Time cost=0.021
INFO:root:Epoch[77] Validation-accuracy=0.985185
INFO:root:Epoch[78] Batch [2]	Speed: 31879.69 samples/sec	accuracy=1.000000
INFO:root:Epoch[78] Batch [4]	Speed: 20673.48 samples/sec	accuracy=1.000000
INFO:root:Epoch[78] Batch [6]	Speed: 22154.96 samples/sec	

...

INFO:root:Epoch[169] Time cost=0.025
INFO:root:Epoch[169] Validation-accuracy=0.981481
INFO:root:Epoch[170] Batch [2]	Speed: 22875.94 samples/sec	accuracy=1.000000
INFO:root:Epoch[170] Batch [4]	Speed: 18955.88 samples/sec	accuracy=1.000000
INFO:root:Epoch[170] Batch [6]	Speed: 19697.73 samples/sec	accuracy=1.000000
INFO:root:Epoch[170] Batch [8]	Speed: 11981.44 samples/sec	accuracy=1.000000
INFO:root:Epoch[170] Train-accuracy=1.000000
INFO:root:Epoch[170] Time cost=0.024
INFO:root:Epoch[170] Validation-accuracy=0.981481
INFO:root:Epoch[171] Batch [2]	Speed: 20098.89 samples/sec	accuracy=1.000000
INFO:root:Epoch[171] Batch [4]	Speed: 19662.34 samples/sec	accuracy=1.000000
INFO:root:Epoch[171] Batch [6]	Speed: 19371.74 samples/sec	accuracy=1.000000
INFO:root:Epoch[171] Batch [8]	Speed: 15903.58 samples/sec	accuracy=1.000000
INFO:root:Epoch[171] Train-accuracy=1.000000
INFO:root:Epoch[171] Time cost=0.026
INFO:root:Epoch[171] Validation-accuracy=0.981481
INFO:root:Epoch[172] Batch [2]	Sp

INFO:root:Epoch[188] Batch [6]	Speed: 17468.99 samples/sec	accuracy=1.000000
INFO:root:Epoch[188] Batch [8]	Speed: 11592.88 samples/sec	accuracy=1.000000
INFO:root:Epoch[188] Train-accuracy=1.000000
INFO:root:Epoch[188] Time cost=0.025
INFO:root:Epoch[188] Validation-accuracy=0.981481
INFO:root:Epoch[189] Batch [2]	Speed: 21955.88 samples/sec	accuracy=1.000000
INFO:root:Epoch[189] Batch [4]	Speed: 17353.35 samples/sec	accuracy=1.000000
INFO:root:Epoch[189] Batch [6]	Speed: 18394.73 samples/sec	accuracy=1.000000
INFO:root:Epoch[189] Batch [8]	Speed: 18375.92 samples/sec	accuracy=1.000000
INFO:root:Epoch[189] Train-accuracy=1.000000
INFO:root:Epoch[189] Time cost=0.023
INFO:root:Epoch[189] Validation-accuracy=0.981481
INFO:root:Epoch[190] Batch [2]	Speed: 22519.75 samples/sec	accuracy=1.000000
INFO:root:Epoch[190] Batch [4]	Speed: 18142.76 samples/sec	accuracy=1.000000
INFO:root:Epoch[190] Batch [6]	Speed: 16481.65 samples/sec	accuracy=1.000000
INFO:root:Epoch[190] Batch [8]	Speed: 16950

INFO:root:Epoch[207] Batch [2]	Speed: 20910.53 samples/sec	accuracy=1.000000
INFO:root:Epoch[207] Batch [4]	Speed: 16487.04 samples/sec	accuracy=1.000000
INFO:root:Epoch[207] Batch [6]	Speed: 15653.31 samples/sec	accuracy=1.000000
INFO:root:Epoch[207] Batch [8]	Speed: 11763.03 samples/sec	accuracy=1.000000
INFO:root:Epoch[207] Train-accuracy=1.000000
INFO:root:Epoch[207] Time cost=0.026
INFO:root:Epoch[207] Validation-accuracy=0.981481
INFO:root:Epoch[208] Batch [2]	Speed: 18255.95 samples/sec	accuracy=1.000000
INFO:root:Epoch[208] Batch [4]	Speed: 15139.16 samples/sec	accuracy=1.000000
INFO:root:Epoch[208] Batch [6]	Speed: 18111.42 samples/sec	accuracy=1.000000
INFO:root:Epoch[208] Batch [8]	Speed: 13245.87 samples/sec	accuracy=1.000000
INFO:root:Epoch[208] Train-accuracy=1.000000
INFO:root:Epoch[208] Time cost=0.027
INFO:root:Epoch[208] Validation-accuracy=0.981481
INFO:root:Epoch[209] Batch [2]	Speed: 21566.39 samples/sec	accuracy=1.000000
INFO:root:Epoch[209] Batch [4]	Speed: 16954

INFO:root:Epoch[225] Train-accuracy=1.000000
INFO:root:Epoch[225] Time cost=0.023
INFO:root:Epoch[225] Validation-accuracy=0.981481
INFO:root:Epoch[226] Batch [2]	Speed: 21172.66 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [4]	Speed: 18791.68 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [6]	Speed: 17975.59 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [8]	Speed: 12606.87 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Train-accuracy=1.000000
INFO:root:Epoch[226] Time cost=0.024
INFO:root:Epoch[226] Validation-accuracy=0.981481
INFO:root:Epoch[227] Batch [2]	Speed: 19625.54 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [4]	Speed: 18981.61 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [6]	Speed: 18195.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [8]	Speed: 12696.55 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Train-accuracy=1.000000
INFO:root:Epoch[227] Time cost=0.025
INFO:root:Epoch[227] Validation-accura

INFO:root:Epoch[244] Batch [6]	Speed: 17884.89 samples/sec	accuracy=1.000000
INFO:root:Epoch[244] Batch [8]	Speed: 16030.21 samples/sec	accuracy=1.000000
INFO:root:Epoch[244] Train-accuracy=1.000000
INFO:root:Epoch[244] Time cost=0.024
INFO:root:Epoch[244] Validation-accuracy=0.981481
INFO:root:Epoch[245] Batch [2]	Speed: 28346.28 samples/sec	accuracy=1.000000
INFO:root:Epoch[245] Batch [4]	Speed: 21457.90 samples/sec	accuracy=1.000000
INFO:root:Epoch[245] Batch [6]	Speed: 20435.10 samples/sec	accuracy=1.000000
INFO:root:Epoch[245] Batch [8]	Speed: 16167.17 samples/sec	accuracy=1.000000
INFO:root:Epoch[245] Train-accuracy=1.000000
INFO:root:Epoch[245] Time cost=0.022
INFO:root:Epoch[245] Validation-accuracy=0.981481
INFO:root:Epoch[246] Batch [2]	Speed: 27934.09 samples/sec	accuracy=1.000000
INFO:root:Epoch[246] Batch [4]	Speed: 21115.81 samples/sec	accuracy=1.000000
INFO:root:Epoch[246] Batch [6]	Speed: 19957.04 samples/sec	accuracy=1.000000
INFO:root:Epoch[246] Batch [8]	Speed: 16015

In [68]:
# RoI-level feature extraction

from bbox.bbox_transform import bbox_pred, clip_boxes

def extract_roi_feats(feat_extractor, mx_data, im_infos):
    roi_feats = []
    bboxes = []
    obj_scores = []

    print("Extract RoI features...")
    tic()

    for idx, (d, im_info) in enumerate(zip(mx_data, im_infos)):
        # batch of size 1
        x = mx.io.DataBatch(data=[d], label=[], pad=0, index=idx,
                            provide_data=[[('data', d[0].shape), ('im_info', d[1].shape)]],
                            provide_label=[None])

        # global average pooling of RoI-pooled conv features
        feat_extractor.forward(x)
        rois, scores, bbox_delta, subcls_scores, conv5_feat = feat_extractor.get_outputs()
        roipooled_conv5_feats = mx.ndarray.ROIPooling(data=conv5_feat, rois=rois, pooled_size=(7, 7), spatial_scale=0.0625)
        pooled_feats = mx.ndarray.Pooling(data=roipooled_conv5_feats, pool_type='avg', global_pool=True, kernel=(7, 7))
        roi_feats.append(pooled_feats.reshape((pooled_feats.shape[0], -1)).asnumpy())

        im_h, im_w, im_scale = im_info[0]
        im_bboxes = bbox_pred(rois.asnumpy().reshape((-1, 5))[:, 1:], np.array([0.1, 0.1, 0.2, 0.2]) * bbox_delta.asnumpy()[0])
        im_bboxes = clip_boxes(im_bboxes, [im_h, im_w])
        im_bboxes = im_bboxes / im_scale
        bboxes.append(im_bboxes)
        obj_scores.append(scores.asnumpy()[:, ])

        if (idx + 1) % 100 == 0:
            print('RoI feature extraction {}/{}: {:.4f} seconds spent.'.format(idx + 1, len(mx_data), toc()))

    print('Done')
    return roi_feats, bboxes, obj_scores

roi_feats, bboxes, obj_scores = extract_roi_feats(feat_extractor, mx_data, im_infos)

Extract RoI features...
RoI feature extraction 100/542: 7.9390 seconds spent.
RoI feature extraction 200/542: 15.4364 seconds spent.
RoI feature extraction 300/542: 21.5099 seconds spent.
RoI feature extraction 400/542: 28.2718 seconds spent.
RoI feature extraction 500/542: 35.4713 seconds spent.
Done


In [69]:
# classify RoIs

def classify_rois(linear_classifier, roi_feats):
    cls_scores = []

    print("Classify RoIs...")
    tic()

    for idx, im_roi_feats in enumerate(roi_feats):
        iterator = mx.io.NDArrayIter([im_roi_feats.reshape((im_roi_feats.shape[0], -1))],
                                     np.zeros((im_roi_feats.shape[0])),
                                     min(100, im_roi_feats.shape[0]), shuffle=False)

        im_cls_scores = linear_classifier.predict(iterator)
        cls_scores.append(im_cls_scores.asnumpy())

        if (idx + 1) % 100 == 0:
            print('RoI classification {}/{}: {:.4f} seconds spent.'.format(idx + 1, len(roi_feats), toc()))

    print('Done')
    return cls_scores

cls_scores = classify_rois(linear_classifier, roi_feats)

Classify RoIs...
RoI classification 100/542: 0.4866 seconds spent.
RoI classification 200/542: 0.9243 seconds spent.
RoI classification 300/542: 1.3667 seconds spent.
RoI classification 400/542: 1.7973 seconds spent.
RoI classification 500/542: 2.1949 seconds spent.
Done


In [70]:
# calculate final scores and perform non-maximum suppression (NMS)

from demo.nms import nms

def process_mul_scores(obj_scores, cls_scores):
    """
    Do multiplication of objectness score and classification score to obtain the final detection score.
    """
    final_scores = np.zeros((obj_scores.shape[0], cls_scores.shape[1] + 1))
    final_scores[:, 1:] = cls_scores[:, :] * obj_scores[:, [1]]
    return final_scores

def postprocess_rois(bboxes, obj_scores, cls_scores, nms_thresh, score_thresh):
    detections = []

    print("Postprocess...")
    tic()

    for idx, (im_bboxes, im_obj_scores, im_cls_scores) in enumerate(zip(bboxes, obj_scores, cls_scores)):
        im_scores = process_mul_scores(im_obj_scores[0], im_cls_scores).astype('f')
        im_bboxes = im_bboxes.astype('f')
        im_detections = []

        for c in range(1, im_scores.shape[1]):
            scores_c = im_scores[:, c, np.newaxis]
            boxes_c = im_bboxes[:, 0:4]
            detections_c = np.hstack((boxes_c, scores_c))

            keep = nms(detections_c, nms_thresh)
            detections_c = detections_c[keep, :]
            detections_c = detections_c[detections_c[:, -1] > float(score_thresh), :]

            im_detections.append(detections_c)
        detections.append(im_detections)

        if (idx + 1) % 100 == 0:
            print('Postprocess {}/{}: {:.4f} seconds spent.'.format(idx + 1, len(bboxes), toc()))

    print('Done')
    return detections

nms_thresh = 0.45
score_thresh = 0.5

detections = postprocess_rois(bboxes, obj_scores, cls_scores, nms_thresh, score_thresh)

Postprocess...
Postprocess 100/542: 1.4520 seconds spent.
Postprocess 200/542: 2.9335 seconds spent.
Postprocess 300/542: 4.1660 seconds spent.
Postprocess 400/542: 5.6460 seconds spent.
Postprocess 500/542: 6.9863 seconds spent.
Done


In [85]:
# annotate bounding-boxes to uploaded data in Superb AI Suite

def annotate_bbox(project, ref_spb_data, detections, class_names):
    command = spb.Command(type='update_label')

    for idx, (d, im_detections) in enumerate(zip(ref_spb_data, detections)):
        objects = []
        obj_idx = 0

        for class_name, detections_c in zip(class_names, im_detections):
            for x1, y1, x2, y2, score in detections_c:
                x = int(round(x1))
                y = int(round(y1))
                w = int(round(x2 - x1))
                h = int(round(y2 - y1))

                objects.append({'id': obj_idx + 1, 'class': class_name,
                                'shape': {'box': {'x': x, 'y': y, 'width': w, 'height': h}},
                                "properties": []})
                obj_idx += 1

        try:
            if d.result is not None:
                d_result = d.result
                d_result.update({'objects': objects})
            else:
                d_result = {'objects': objects}
            d.result = d_result
            d = spb.run(command=command, option=d)
            print('Annotate {}/{}: {} boxes -> {}'.format(idx + 1, len(ref_spb_data), len(objects), d.data_key))
        except Exception as e:
            print('[Exception {}] Annotate {}/{}: {} boxes -> {}'.format(str(e), idx + 1, len(ref_spb_data), len(objects), d.data_key))

    print('Done')

annotate_bbox(project, ref_spb_data, detections, class_names)

Annotate 1/542: 1 boxes -> train_SDK_OD_Test_bugatti_78.jpg
Annotate 2/542: 1 boxes -> train_SDK_OD_Test_bugatti_3.jpg
Annotate 3/542: 3 boxes -> train_SDK_OD_Test_bugatti_63.jpg
Annotate 4/542: 1 boxes -> train_SDK_OD_Test_bugatti_1.jpg
Annotate 5/542: 1 boxes -> train_SDK_OD_Test_bugatti_84.jpg
Annotate 6/542: 1 boxes -> train_SDK_OD_Test_bugatti_18.jpg
Annotate 7/542: 2 boxes -> train_SDK_OD_Test_bugatti_29.jpg
Annotate 8/542: 1 boxes -> train_SDK_OD_Test_bugatti_64.jpg
Annotate 9/542: 1 boxes -> train_SDK_OD_Test_bugatti_98.jpg
Annotate 10/542: 0 boxes -> train_SDK_OD_Test_bugatti_111.jpg
Annotate 11/542: 1 boxes -> train_SDK_OD_Test_bugatti_83.jpg
Annotate 12/542: 1 boxes -> train_SDK_OD_Test_bugatti_32.jpg
Annotate 13/542: 1 boxes -> train_SDK_OD_Test_bugatti_46.jpg
Annotate 14/542: 1 boxes -> train_SDK_OD_Test_bugatti_12.jpg
Annotate 15/542: 1 boxes -> train_SDK_OD_Test_bugatti_85.jpg
Annotate 16/542: 1 boxes -> train_SDK_OD_Test_bugatti_67.jpg
Annotate 17/542: 1 boxes -> train_

Annotate 136/542: 1 boxes -> train_SDK_OD_Test_gpu_92.jpg
Annotate 137/542: 1 boxes -> train_SDK_OD_Test_gpu_65.jpg
Annotate 138/542: 1 boxes -> train_SDK_OD_Test_gpu_70.jpg
Annotate 139/542: 1 boxes -> train_SDK_OD_Test_gpu_24.jpg
Annotate 140/542: 1 boxes -> train_SDK_OD_Test_gpu_69.jpg
Annotate 141/542: 1 boxes -> train_SDK_OD_Test_gpu_48.jpg
Annotate 142/542: 1 boxes -> train_SDK_OD_Test_gpu_87.jpg
Annotate 143/542: 1 boxes -> train_SDK_OD_Test_gpu_9.jpg
Annotate 144/542: 1 boxes -> train_SDK_OD_Test_gpu_113.jpg
Annotate 145/542: 1 boxes -> train_SDK_OD_Test_gpu_68.jpg
Annotate 146/542: 1 boxes -> train_SDK_OD_Test_gpu_89.jpg
Annotate 147/542: 1 boxes -> train_SDK_OD_Test_gpu_85.jpg
Annotate 148/542: 1 boxes -> train_SDK_OD_Test_gpu_106.jpg
Annotate 149/542: 1 boxes -> train_SDK_OD_Test_gpu_88.jpg
Annotate 150/542: 2 boxes -> train_SDK_OD_Test_gpu_46.jpg
Annotate 151/542: 1 boxes -> train_SDK_OD_Test_gpu_42.jpg
Annotate 152/542: 1 boxes -> train_SDK_OD_Test_gpu_72.jpg
Annotate 153/

Annotate 272/542: 1 boxes -> train_SDK_OD_Test_spiderman_65.jpg
Annotate 273/542: 1 boxes -> train_SDK_OD_Test_spiderman_17.jpg
Annotate 274/542: 2 boxes -> train_SDK_OD_Test_spiderman_49.jpg
Annotate 275/542: 1 boxes -> train_SDK_OD_Test_spiderman_5.jpg
Annotate 276/542: 1 boxes -> train_SDK_OD_Test_spiderman_37.jpg
Annotate 277/542: 1 boxes -> train_SDK_OD_Test_spiderman_79.jpg
Annotate 278/542: 1 boxes -> train_SDK_OD_Test_spiderman_74.jpg
Annotate 279/542: 2 boxes -> train_SDK_OD_Test_spiderman_1.jpg
Annotate 280/542: 1 boxes -> train_SDK_OD_Test_spiderman_11.jpg
Annotate 281/542: 1 boxes -> train_SDK_OD_Test_spiderman_4.jpg
Annotate 282/542: 1 boxes -> train_SDK_OD_Test_spiderman_90.jpg
Annotate 283/542: 2 boxes -> train_SDK_OD_Test_spiderman_35.jpg
Annotate 284/542: 1 boxes -> train_SDK_OD_Test_spiderman_59.jpg
Annotate 285/542: 2 boxes -> train_SDK_OD_Test_spiderman_16.jpg
Annotate 286/542: 0 boxes -> train_SDK_OD_Test_spiderman_12.jpg
Annotate 287/542: 1 boxes -> train_SDK_OD_T

Annotate 409/542: 1 boxes -> val_SDK_OD_Test_gpu_34.jpg
Annotate 410/542: 1 boxes -> val_SDK_OD_Test_gpu_52.jpg
Annotate 411/542: 1 boxes -> val_SDK_OD_Test_gpu_54.jpg
Annotate 412/542: 1 boxes -> val_SDK_OD_Test_gpu_62.jpg
Annotate 413/542: 1 boxes -> val_SDK_OD_Test_gpu_97.jpg
Annotate 414/542: 1 boxes -> val_SDK_OD_Test_gpu_117.jpg
Annotate 415/542: 1 boxes -> val_SDK_OD_Test_gpu_40.jpg
Annotate 416/542: 1 boxes -> val_SDK_OD_Test_gpu_107.jpg
Annotate 417/542: 1 boxes -> val_SDK_OD_Test_gpu_53.jpg
Annotate 418/542: 1 boxes -> val_SDK_OD_Test_gpu_81.jpg
Annotate 419/542: 2 boxes -> val_SDK_OD_Test_gpu_105.jpg
Annotate 420/542: 1 boxes -> val_SDK_OD_Test_gpu_66.jpg
Annotate 421/542: 1 boxes -> val_SDK_OD_Test_gpu_5.jpg
Annotate 422/542: 1 boxes -> val_SDK_OD_Test_gpu_37.jpg
Annotate 423/542: 1 boxes -> val_SDK_OD_Test_gpu_75.jpg
Annotate 424/542: 1 boxes -> val_SDK_OD_Test_gpu_0.jpg
Annotate 425/542: 1 boxes -> val_SDK_OD_Test_gpu_61.jpg
Annotate 426/542: 2 boxes -> val_SDK_OD_Test_gp